<a href="https://colab.research.google.com/github/oburckard/MyRepo/blob/master/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

4*x1 − x2 + x3 = 8

2*x1 + 5*x2 + 2*x3 = 3

x1 + 2*x2 + 4*x3 = 11

=> 4*x1 − x2 + x3 = 8

   11*x2 + 3*x3 = -2

   9*x2 + 15*x3 = 36

=> 4*x1 − x2 + x3 = 8

   46*x2 = -46

   9*x2 + 15*x3 = 36

=> 4*x1 − x2 + x3 = 8

   x2 = -1
   x3 = 3


=> x1 = 1

   x2 = -1

   x3 = 3

------------------------

4*x1 − x2 + 2*x3 = 9

2*x1 + 4*x2 − x3 = −5

x1 + x2 − 3*x3 = −9

=> -5*x2 + 14*x3 = 45

   2*x2 + 5*x3 = 13

   x1 + x2 − 3*x3 = −9

=> -5*x2 + 14*x3 = 45 

   (53/5) * x3 = 31

   x1 + x2 − 3*x3 = −9

=> x3 = 155/53

   x2 = -43/53
   
   x1 = 31/53   

In [0]:
# Gaussian elimination

import scipy 
import scipy.linalg
import numpy as np
import copy

def linearsolver(A,b):
  n = len(A)
  M = A

  i = 0
  for x in M:
   x.append(b[i])
   i += 1


  for k in range(n):
   # first pivoting
   for i in range(k,n):
     if abs(M[i][k]) > abs(M[k][k]):
        M[k], M[i] = M[i],M[k]
     else:
        pass
   
   # Gaussian elimination in loop 
   for j in range(k+1,n):
       q = float(M[j][k]) / M[k][k]
       for m in range(k, n+1):
          # M_jm = M_jm - M_jk / M_kk *M_km
          M[j][m] -=  q * M[k][m]

  x = [0 for i in range(n)]       # x = [0, 0, .... , 0]

  # calculating x
  x[n-1] =float(M[n-1][n])/M[n-1][n-1]
  for i in range (n-1,-1,-1):
    z = 0
    for j in range(i+1,n):
        z = z  + float(M[i][j])*x[j]
    x[i] = float(M[i][n] - z)/M[i][i]
  return x


#A = [[2, 1, 1, 0], [4, 3, 3, 1], [8, 7, 9, 5], [6, 7, 9, 8]]
#b = [1, 2, 3, 4]

A = [[4, -1, 1], [2, 5, 2], [1, 2, 4]]
b = [8, 3, 11]

#A = [[4, -1, 2], [2, 4, -1], [1, 1, -3]]
#b = [9, -5, -9]

A1 = copy.deepcopy(A)
x = linearsolver(A1, b)

print("Result with our linearsolver:")
print(np.array(A)," * x = ",np.array(b))
print("Solution:")
print(x)




x = scipy.linalg.solve(A,b)

print(" ")
print("Result with linalg.solve:")
print(np.array(A)," * x = ",np.array(b))
print("Solution:")
print(x)

Result with our linearsolver:
[[ 4 -1  1]
 [ 2  5  2]
 [ 1  2  4]]  * x =  [ 8  3 11]
Solution:
[1.0, -1.0, 3.0]
 
Result with linalg.solve:
[[ 4 -1  1]
 [ 2  5  2]
 [ 1  2  4]]  * x =  [ 8  3 11]
Solution:
[ 1. -1.  3.]


In [0]:
# LU decomposition

import numpy as np

def LU_partial_decomposition(matrix,b):
    n, m = matrix.shape
    P = np.identity(n)
    L = np.identity(n)
    U = matrix.copy()
    bb = b.copy()
    PF = np.identity(n)
    LF = np.zeros((n,n))
    for k in range(0, n - 1):
        index = np.argmax(abs(U[k:,k]))
        index = index + k 
        
        # here we do pivoting
        if index != k:
            P = np.identity(n)
            P[[index,k],k:n] = P[[k,index],k:n]
            U[[index,k],k:n] = U[[k,index],k:n]
            bb[k] = b[index]
            bb[index] = b[k]
            b = bb.copy()
            PF = np.dot(P,PF)
            LF = np.dot(P,LF)
            
        # and here decomposition    
        L = np.identity(n)
        for j in range(k+1,n):
            L[j,k] = -(U[j,k] / U[k,k])
            LF[j,k] = (U[j,k] / U[k,k])
        U = np.dot(L,U)
    np.fill_diagonal(LF, 1)
  
    return PF, LF, U, bb


def solve_Ly(L,b):
  n, n = L.shape
  y = np.zeros(n)
  y[0] = b[0]/L[0,0]
  for k in range(1,n):
    y[k] = b[k]
    for i in range(0,k):
      y[k] = y[k] - L[k,i] * y[i]
    y[k] = y[k] / L[k,k]
    
  return y  



def solve_Ux(U,y):
  n, n = U.shape
  x = np.zeros(n)
  x[n-1] = y[n-1]/U[n-1,n-1]
  for k in range(n-2,-1,-1):
    x[k] = y[k]
    for i in range(n-1,k,-1):
       x[k] = x[k] - U[k,i] * x[i]
    x[k] = x[k] / U[k,k]

  return x 
      


# Usage
#A = [[2, 1, 1, 0], [4, 3, 3, 1], [8, 7, 9, 5], [6, 7, 9, 8]]
#b = [1, 2, 3, 4]
A = [[1,2,1], [1,-2,2], [2,12,-2]]
b = [0,4,4]
#A = [[4, -1, 1], [2, 5, 2], [1, 2, 4]]
#b = [8, 3, 11]
#A = [[4, -1, 2], [2, 4, -1], [1, 1, -3]]
#b = [9, -5, -9]
A = np.array(A)
P1, L1, U1, bb = LU_partial_decomposition(A,b)
y = solve_Ly(L1,bb)
x = solve_Ux(U1,y)

# Scipy
import scipy
import scipy.linalg
#LU decomposition using linalg.lu library function
P2, L2, U2 = scipy.linalg.lu(A) # P2 = P2.T  

print("Input array:")
print(A)
print(" ")
print("Result. Pivoting array:")
print(P1)
print("L and U arrays:")
print(L1,"\n", U1)
print(" ")
print("Result using linalg.lu function. Pivoting array:")
print(P2.T)  # transponed array
print("L and U arrays:")
print(L2,"\n", U2)
print('  ')
print("Results :")
print("y:", y)
print("x :",x)

Input array:
[[ 1  2  1]
 [ 1 -2  2]
 [ 2 12 -2]]
 
Result. Pivoting array:
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
L and U arrays:
[[1.  0.  0. ]
 [0.5 1.  0. ]
 [0.5 0.5 1. ]] 
 [[ 2.  12.  -2. ]
 [ 0.  -8.   3. ]
 [ 0.   0.   0.5]]
 
Result using linalg.lu function. Pivoting array:
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
L and U arrays:
[[1.  0.  0. ]
 [0.5 1.  0. ]
 [0.5 0.5 1. ]] 
 [[ 2.  12.  -2. ]
 [ 0.  -8.   3. ]
 [ 0.   0.   0.5]]
  
Results :
y: [ 4.  2. -3.]
x : [11.  -2.5 -6. ]
